# 1. Importing Libs

In [ ]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

# physical_devices = tf.config.list_physical_devices("GPU")
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

# 2. Importing data

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print("Before")
print(type(x_train))
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)


x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

print("After")
print(type(x_train))
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

11490434/11490434 [==============================] - 0s 0us/step
Before
<class 'numpy.ndarray'>
(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)
After
<class 'numpy.ndarray'>
(60000, 28, 28, 1)
(60000,)
(10000, 28, 28, 1)
(10000,)


## 2.1 Using classes to combine multiple layers into a single CNN Block that can be used multiple times

References:
[What is the difference between __init__ and __call__](https://stackoverflow.com/questions/9663562/what-is-the-difference-between-init-and-call#:~:text=So%2C%20__init__%20is,initializing%20the%20instance%20variable%20also.&text=And%20__call__%20is,object%20like%20any%20other%20function.&text=Save%20this%20answer.,-Show%20activity%20on)

In [ ]:
# CNN -> BatchNorm -> ReLU (common structure)
# x10 (a lot of code to write!)


class CNNBlock(layers.Layer):
    def __init__(self, out_channels, kernel_size=3):
        super(CNNBlock, self).__init__()
        self.conv = layers.Conv2D(out_channels, kernel_size, padding="same")
        self.bn = layers.BatchNormalization()

    def call(self, input_tensor, training=False):
        x = self.conv(input_tensor)
        x = self.bn(x, training=training)
        x = tf.nn.relu(x)
        return x


model = keras.Sequential(
    [
        CNNBlock(32),
        CNNBlock(64),
        CNNBlock(128),
        layers.Flatten(),
        layers.Dense(10)
    ]
)
# print(model.summary())

# model.compile(
#     optimizer=keras.optimizers.Adam(),
#     loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#     metrics=["accuracy"],
# )

# model.fit(x_train, y_train, batch_size=64, epochs=2, verbose=2)
# model.evaluate(x_test, y_test, batch_size=64, verbose=2)

In [ ]:
class ResBlock(layers.Layer):
    def __init__(self, channels):
        super(ResBlock, self).__init__()
        self.channels = channels
        self.cnn1 = CNNBlock(channels[0], 3)
        self.cnn2 = CNNBlock(channels[1], 3)
        self.cnn3 = CNNBlock(channels[2], 3)
        self.pooling = layers.MaxPooling2D()
        self.identity_mapping = layers.Conv2D(channels[1], 1, padding="same")

    def call(self, input_tensor, training=False):
        x = self.cnn1(input_tensor, training=training)
        x = self.cnn2(x, training=training)
        x = self.cnn3(x + self.identity_mapping(input_tensor), training=training,)
        x = self.pooling(x)
        return x


class ResNet_Like(keras.Model):
    def __init__(self, num_classes=10):
        super(ResNet_Like, self).__init__()
        self.block1 = ResBlock([32, 32, 64])
        self.block2 = ResBlock([128, 128, 256])
        self.block3 = ResBlock([128, 256, 512])
        self.pool = layers.GlobalAveragePooling2D()
        self.classifier = layers.Dense(num_classes)

    def call(self, input_tensor, training=False):
        x = self.block1(input_tensor, training=training)
        x = self.block2(x, training=training)
        x = self.block3(x, training=training)
        x = self.pool(x, training=training)
        x = self.classifier(x)
        return x

    def model(self):
        x = keras.Input(shape=(28, 28, 1))
        return keras.Model(inputs=[x], outputs=self.call(x))

In [ ]:
# # model = ResNet_Like().model()
# # base_input = model.layers[0].input
# # base_output = model.layers[2].output
# # output = layers.Dense(10)(layers.Flatten()(base_output))
# model = ResNet_Like(num_classes=10).model()
# # model = keras.Model(base_input, output)

# model.compile(
#     optimizer=keras.optimizers.Adam(),
#     loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#     metrics=["accuracy"],
# )

# model.fit(x_train, y_train, batch_size=64, epochs=2, verbose=2)
# model.evaluate(x_test, y_test, batch_size=64, verbose=2)
# model.save("pretrained")

In [ ]:
model = ResNet_Like().model()
base_input = model.layers[0].input
base_output = model.layers[2].output
print(base_input)
print(base_output)
output = layers.Dense(10)(layers.Flatten()(base_output))
model = keras.Model(base_input, output)

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

model.fit(x_train, y_train, batch_size=64, epochs=1, verbose=2)
model.evaluate(x_test, y_test, batch_size=64, verbose=2)
# model.save("pretrained")

KerasTensor(type_spec=TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 7, 7, 256), dtype=tf.float32, name=None), name='res_block_1/max_pooling2d_1/MaxPool:0', description="created by layer 'res_block_1'")
938/938 - 757s - loss: 0.1048 - accuracy: 0.9684 - 757s/epoch - 807ms/step
157/157 - 33s - loss: 0.0472 - accuracy: 0.9853 - 33s/epoch - 213ms/step


[0.047229740768671036, 0.9853000044822693]